# Yeast cells detection sample notebook

In [1]:
import sys
sys.path.append('../')
#sys.path.append('%HOMEPATH%/detectron2') #insert local path to detecron2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tifffile

import yeastcells.clustering as clustering
import yeastcells.features as features
import yeastcells.data as data
import yeastcells.model as model
import yeastcells.output as output
import yeastcells.visualize as visualize

%matplotlib notebook

In [2]:
cd ~

/home/p301081


### Load data

In [3]:
path = '../../slow-data/yeastcells/data/DATA_CIT/Test_Images'
filenames = data.load_data(path, ff = '.tif')
print(filenames)

['../../slow-data/yeastcells/data/DATA_CIT/Test_Images/Movie_test1.tif']


In [10]:
fn = filenames[0]
start=40 #frame <start> becomes 0 in rest of analysis
image_ = data.read_image(fn, single_im = False, shape=2, start_frame=start, channel=5, grayscale=False)
np.shape(image) #(frames, length, width, channels)

(261, 512, 512, 3)

### Load and run model on data for segmentation

In [6]:
#set path to model_final.pth
model_path = '/var/tensorflow-logs/p253591/yeast-cell-detection-run-4'

#load model
predictor = model.load_model(model_path, seg_thresh=0.94, device='cuda:0') #set GPU if available otherwise use 'cpu'

In [7]:
output = [
  {'instances': predictor(frame)['instances'].to('cpu')}
  for frame in image
]

/home/p301081/.virtualenvs/yeastcells/lib/python3.7/site-packages/detectron2/layers/wrappers.py:226: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  return x.nonzero().unbind(1)


### Run clustering algorithm for tracking

In [8]:
labels, coordinates = clustering.cluster_cells(output, dmax=5, min_samples=5, eps=0.6, progress=False) #for time-series
#labels, coordinates = get_instance_numbers(output) #for single image

/home/p301081/.virtualenvs/yeastcells/lib/python3.7/site-packages/sklearn/neighbors/_base.py:168: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  EfficiencyWarning)


In [ ]:
#plot paths
ax = visualize.plot_paths(
  labels,
  coordinates[:, [2,1,0]],
  style={'linewidth': 1},
  xlim=(0, 512),
  ylim=(0, 512),
  title='',
)

### Extract features and visualize tracking and segmentation

In [16]:
s, t = features.get_seg_track(labels, output) # total

The number of segmentations is 16125
The number of tracked cells is 431


In [17]:
s, t = features.get_seg_track(labels, output, frame = 0) # by frame

The number of segmentations in frame 0 is 93
The number of tracked cells in frame 0 is 66


In [18]:
features.track_len(labels, label=0) #select a label to extract the length of its track

255

In [19]:
contours = features.extract_contours(output) 

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=[255,0,255], framenum=True, labelnum=True)
movie = visualize.show_animation(scene, title='', delay=100) 

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=None, framenum=False, labelnum=True)
cell_scene, label = visualize.select_cell(scene, coordinates, labels, w=60, l=0) # choose image window (w) and label (l)
movie = visualize.show_animation(cell_scene, title='', delay=100)

In [20]:
polygons_clust =  features.polygons_per_cluster(labels, contours, output)

In [ ]:
ax = visualize.plot_area_profiles(polygons_clust, ti=3, label_list=[0,16], ax=None, title="Area in pixels over time")

### Output

In [ ]:
_, _, pred_df = data.get_pred(output, labels, coordinates, ti=3, start=start)

In [21]:
masks = features.get_masks(output)
polygons_inst =  features.polygons_per_instance(labels, contours, output)
poly_area, mask_area = features.get_area(polygons_inst, masks, labels, output, start=start)
pi = features.get_pixel_intesnity(masks, output, image_f)

In [ ]:
pred_df['Poly_Area(pxl)'] = poly_area
pred_df['Mask_Area(pxl)'] = mask_area
pred_df['Pixel_intensity'] = pi
pred_df.head(5)

In [ ]:
visualize.plot_polygon_mask(
    masks, labels, output, image, polygons_clust, 
    label_list=[0,1,2,3,4,5,6,7,9,10], frame=0, ax=None, title=None
)

#### Output for tracked cells

In [24]:
agr = features.get_average_growth_rate(polygons_clust, labels, output)
area_std = features.get_area_std(polygons_clust, labels, pred_df)
position_std = features.get_position_std(polygons_clust, labels, pred_df)

In [25]:
label_df = pd.DataFrame(agr, columns=['Cell_label', 'Average Growth Rate'])
label_df['Std Area(pxl)'] = area_std[:,1]
label_df['Std Position'] = position_std[:,1]

In [ ]:
label_df['Cell_label'] = label_df['Cell_label'].astype(int)
label_df.head(5)

### Outputs

In [ ]:
#Dataframe into csv
# pred_df = pred_df.sort_values(['Cell_label', 'Frame_number'])  
# pred_df = pred_df.reset_index(drop=True)
df_out = 'test'
pred_df.to_csv(f'{df_out}.csv', index=False)

In [72]:
#masks as tiff stack
masks_out = 'test'
with tifffile.TiffWriter(f'{masks_out}.tif') as stack:
    for m in masks:
        stack.save(m)